# HT 5. Validación Cruzada

## Predicción de calidad de vinos en base a características físico-químicas

**MINERÍA DE DATOS - SECCIÓN - 20 - 2024 - 1**

Samuel Chamalé, 21881

Renatto Guzmán, 21646

---


## 1. Exploración y Preprocesamiento de Datos


### Cargar el conjunto de datos utilizando Pandas.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ydata_profiling import ProfileReport

In [4]:
# Cargar los datos de vinos rojos y blancos
path_red = "wine+quality/winequality-red.csv"
path_white = "wine+quality/winequality-white.csv"

In [5]:
# Asumimos que el separador en los archivos CSV es ';'
red_wine = pd.read_csv(path_red, sep=';')
white_wine = pd.read_csv(path_white, sep=';')

In [6]:
# Combinar los datasets añadiendo una columna de 'type'
red_wine['type'] = 'red'
white_wine['type'] = 'white'
wine_data = pd.concat([red_wine, white_wine], ignore_index=True)

## Realizar un análisis exploratorio de datos (EDA) básico: estadísticas descriptivas,

distribución de variables, etc.
